# Clustering of Neighborhoods in Toronto

Importing the important modules

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup # to scrape the wiki file

print('Libraries imported.')

Libraries imported.


Using Beatiful Soup to get the information from Wikipedia

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table', class_='wikitable')
tableContents = []

for tableRows in table.find_all('td'):
    tableContents += [tableRows.text.strip()]

Creating a Dataframe of the postal codes, replacing the '/' with ',' and filtering out the 'Not assigned' boroughs

In [3]:
column_names = ['Postal Code', 'Borough', 'Neighborhood'] 
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods
no_of_rows = len(tableContents)
for i in range(0, no_of_rows, 3):
    postal_code = tableContents[i]
    borough = tableContents[i+1]
    neighborhood_name = tableContents[i+2]
    neighborhood_name = neighborhood_name.replace(' /', ',')
    neighborhood_name = neighborhood_name.replace('\n', ',')
    if borough != 'Not assigned':
        neighborhoods = neighborhoods.append({'Postal Code': postal_code,
                                              'Borough': borough,
                                              'Neighborhood': neighborhood_name}, ignore_index=True)
neighborhoods.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
print ('The shape of the dataframe is: ', neighborhoods.shape)

The shape of the dataframe is:  (103, 3)


### Getting the coordinates of the postal codes
Geocoder.google never returned any value and used to freeze whenever it was run, therefore using the provided csv file

In [5]:
location = pd.read_csv('Geospatial_Coordinates.csv')
location.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging the coordinates of the postal codes with the original dataframe

In [6]:
neighborhoods = pd.merge(neighborhoods,
                  location,
                  on='Postal Code')
neighborhoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [7]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


Getting the latitude and longitude of Toronto using geolocator

In [8]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Creating the map of Toronto

In [9]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Declaring my Foursquare credentials

In [10]:
CLIENT_ID = 'ZEFZ0PFM0IGN4ZPNGADEA0NGRWPQH40HCPKQ0EQCHT52JMSO' # your Foursquare ID
CLIENT_SECRET = 'X5XZUTMI0KKMZUR5TTBQDBB055TZDQ0GRYSQYIQONAX1EO2T' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZEFZ0PFM0IGN4ZPNGADEA0NGRWPQH40HCPKQ0EQCHT52JMSO
CLIENT_SECRET:X5XZUTMI0KKMZUR5TTBQDBB055TZDQ0GRYSQYIQONAX1EO2T


## Analyzing one Neighborhood

In [11]:
neighborhoods.loc[0, 'Neighborhood']
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


### Geting the venues near the neighborhood with a LIMIT of 100 and a radius of 1500 meters

In [12]:
LIMIT = 100
radius = 1500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=ZEFZ0PFM0IGN4ZPNGADEA0NGRWPQH40HCPKQ0EQCHT52JMSO&client_secret=X5XZUTMI0KKMZUR5TTBQDBB055TZDQ0GRYSQYIQONAX1EO2T&v=20180605&ll=43.7532586,-79.3296565&radius=1500&limit=100'

Results of the api call

In [13]:
results = requests.get(url).json()

Cleaning the json and structing it into a pandas dataframe

In [14]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Allwyn's Bakery,Caribbean Restaurant,43.759840,-79.324719
1,Donalda Golf & Country Club,Golf Course,43.752816,-79.342741
2,Brookbanks Park,Park,43.751976,-79.332140
3,Tim Hortons,Café,43.760668,-79.326368
4,LCBO,Liquor Store,43.757774,-79.314257


In [16]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

38 venues were returned by Foursquare.


## Exploring all the neighborhoods in Toronto

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview
The Danforth West, Ri

In [19]:
print(toronto_venues.shape)
toronto_venues.head()

(6909, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
2,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
3,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
4,Parkwoods,43.753259,-79.329656,LCBO,43.757774,-79.314257,Liquor Store


Number of venues returned in each location

In [20]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,60,60,60,60,60,60
"Alderwood, Long Branch",46,46,46,46,46,46
"Bathurst Manor, Wilson Heights, Downsview North",40,40,40,40,40,40
Bayview Village,15,15,15,15,15,15
"Bedford Park, Lawrence Manor East",77,77,77,77,77,77
Berczy Park,100,100,100,100,100,100
"Birch Cliff, Cliffside West",14,14,14,14,14,14
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
Business reply mail Processing CentrE,100,100,100,100,100,100


Number of venue category returned

In [21]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 346 uniques categories.


# Analyzing each Neighborhood

In [22]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# # move neighborhood column to the first column
# fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
# toronto_onehot = toronto_onehot[fixed_columns]

move = toronto_onehot['Neighborhood']
toronto_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
toronto_onehot.insert(0, 'Neighborhood', move)

toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Library,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculptu

In [23]:
print('The shape of the dataframe is: ', toronto_onehot.shape)

The shape of the dataframe is:  (6909, 346)


Grouping the Neighborhood and taking the mean of the frequencies in each category

In [24]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Library,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculptu

In [25]:
print('The shape of the dataframe is: ', toronto_grouped.shape)

The shape of the dataframe is:  (98, 346)


Printing the top 5 most common venues in each neighborhood

In [26]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

 Place  0.04
2  Clothing Store  0.04
3            Park  0.04
4     Coffee Shop  0.04


----Bathurst Manor, Wilson Heights, Downsview North----
         venue  freq
0         Park  0.12
1  Coffee Shop  0.08
2  Pizza Place  0.05
3  Gas Station  0.05
4         Bank  0.05


----Bayview Village----
           venue  freq
0           Bank  0.13
1    Gas Station  0.13
2           Park  0.13
3          Trail  0.13
4  Grocery Store  0.07


----Bedford Park, Lawrence Manor East----
                venue  freq
0              Bakery  0.10
1  Italian Restaurant  0.08
2    Sushi Restaurant  0.06
3         Coffee Shop  0.06
4          Bagel Shop  0.05


----Berczy Park----
                 venue  freq
0          Coffee Shop  0.15
1                 Café  0.06
2           Restaurant  0.05
3                Hotel  0.05
4  Japanese Restaurant  0.04


----Birch Cliff, Cliffside West----
                   venue  freq
0                   Park  0.14
1        Thai Restaurant  0.07
2                  Diner  0.

Putting that information into the pandas dataframe displaying the top 10 venues for each neighborhood

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Coffee Shop,Bakery,Shopping Mall,Cantonese Restaurant,Breakfast Spot,Caribbean Restaurant,Gym / Fitness Center,Pizza Place,Lounge
1,"Alderwood, Long Branch",Pharmacy,Park,Burger Joint,Café,Discount Store,Bank,Grocery Store,Toy / Game Store,Coffee Shop,Clothing Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Park,Coffee Shop,Bank,Pizza Place,Gas Station,Restaurant,Bus Line,Dog Run,Sandwich Place,Fried Chicken Joint
3,Bayview Village,Bank,Park,Trail,Gas Station,Intersection,Chinese Restaurant,Café,Restaurant,Skating Rink,Grocery Store
4,"Bedford Park, Lawrence Manor East",Bakery,Italian Restaurant,Coffee Shop,Sushi Restaurant,Bagel Shop,Pizza Place,Café,Sandwich Place,Restaurant,Fast Food Restaurant


# Clustering the Neighborhoods

In [29]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# add clustering labels/ drop if it already exists
if 'Cluster Labels' in neighborhoods_venues_sorted:
    neighborhoods_venues_sorted = neighborhoods_venues_sorted.drop('Cluster Labels', 1)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with neighborhoods to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3,Pharmacy,Coffee Shop,Park,Supermarket,Gas Station,Intersection,Bank,Pizza Place,Golf Course,Fish & Chips Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,3,Coffee Shop,Gym,Middle Eastern Restaurant,Grocery Store,Fast Food Restaurant,Gym / Fitness Center,Mediterranean Restaurant,Bank,Beer Store,Bookstore
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Café,Restaurant,Park,Pub,Farmers Market,Bar,Gastropub,Japanese Restaurant,Italian Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,3,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Sandwich Place,Dessert Shop,Furniture / Home Store,Vietnamese Restaurant,Latin American Restaurant,Men's Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Japanese Restaurant,Park,Café,Gastropub,Dance Studio,Pizza Place,Steakhouse,Bookstore,Bubble Tea Shop


Visualizing with a map

In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examining Each Cluster

### Cluster 1

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
95,Scarborough,0,National Park,Farm,Zoo Exhibit,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant


### Cluster 2

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,York,1,Coffee Shop,Bank,Pizza Place,Italian Restaurant,Café,Indian Restaurant,Bakery,Grocery Store,Restaurant,Caribbean Restaurant
21,York,1,Italian Restaurant,Coffee Shop,Caribbean Restaurant,Furniture / Home Store,Pizza Place,Bakery,Grocery Store,Beer Store,Pharmacy,Sushi Restaurant
23,East York,1,Indian Restaurant,Bakery,Grocery Store,Coffee Shop,Restaurant,Electronics Store,Supermarket,Thai Restaurant,Sporting Goods Shop,Burger Joint
25,Downtown Toronto,1,Café,Coffee Shop,Bar,Korean Restaurant,Italian Restaurant,Park,Ice Cream Shop,Restaurant,Indian Restaurant,Grocery Store
31,West Toronto,1,Café,Italian Restaurant,Coffee Shop,Bar,Bakery,Park,Sushi Restaurant,Breakfast Spot,Sandwich Place,Brazilian Restaurant
35,East York,1,Café,Greek Restaurant,Pizza Place,Coffee Shop,Ice Cream Shop,Thai Restaurant,Bakery,Park,Ethiopian Restaurant,Dessert Shop
37,West Toronto,1,Café,Restaurant,Bar,Italian Restaurant,Cocktail Bar,Bakery,Vegetarian / Vegan Restaurant,Asian Restaurant,Pizza Place,Coffee Shop
41,East Toronto,1,Café,Greek Restaurant,Park,Pub,Pizza Place,Bakery,Coffee Shop,Grocery Store,Vietnamese Restaurant,Flower Shop
43,West Toronto,1,Café,Coffee Shop,Bar,Restaurant,Gift Shop,Bakery,Arts & Crafts Store,Japanese Restaurant,Soccer Stadium,Tea Room
47,East Toronto,1,Coffee Shop,Park,Indian Restaurant,Café,Brewery,Pub,Italian Restaurant,Bakery,Beach,BBQ Joint


### Cluster 3

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Park,Pub,Farmers Market,Bar,Gastropub,Japanese Restaurant,Italian Restaurant
4,Downtown Toronto,2,Coffee Shop,Japanese Restaurant,Park,Café,Gastropub,Dance Studio,Pizza Place,Steakhouse,Bookstore,Bubble Tea Shop
7,North York,2,Coffee Shop,Restaurant,Japanese Restaurant,Bank,Pizza Place,Italian Restaurant,Sandwich Place,Middle Eastern Restaurant,Gym,Park
9,Downtown Toronto,2,Coffee Shop,Japanese Restaurant,Gastropub,Restaurant,Café,Theater,Ramen Restaurant,Burrito Place,Italian Restaurant,Seafood Restaurant
13,North York,2,Coffee Shop,Restaurant,Japanese Restaurant,Bank,Pizza Place,Italian Restaurant,Sandwich Place,Middle Eastern Restaurant,Gym,Park
15,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Hotel,Beer Bar,Gastropub,Japanese Restaurant,Italian Restaurant,Vegetarian / Vegan Restaurant,Clothing Store
19,East Toronto,2,Coffee Shop,Pub,Breakfast Spot,Japanese Restaurant,Grocery Store,Bakery,Beach,Sandwich Place,Pizza Place,BBQ Joint
20,Downtown Toronto,2,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Beer Bar,Park,Farmers Market,Italian Restaurant,American Restaurant
24,Downtown Toronto,2,Café,Park,Restaurant,Ramen Restaurant,Japanese Restaurant,Coffee Shop,Gastropub,Hotel,Mexican Restaurant,Bookstore
30,Downtown Toronto,2,Coffee Shop,Restaurant,Hotel,Café,Japanese Restaurant,Beer Bar,Plaza,Gastropub,Vegetarian / Vegan Restaurant,Theater


### Cluster 4

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,3,Pharmacy,Coffee Shop,Park,Supermarket,Gas Station,Intersection,Bank,Pizza Place,Golf Course,Fish & Chips Shop
1,North York,3,Coffee Shop,Gym,Middle Eastern Restaurant,Grocery Store,Fast Food Restaurant,Gym / Fitness Center,Mediterranean Restaurant,Bank,Beer Store,Bookstore
3,North York,3,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Sandwich Place,Dessert Shop,Furniture / Home Store,Vietnamese Restaurant,Latin American Restaurant,Men's Store
6,Scarborough,3,Zoo Exhibit,Fast Food Restaurant,Trail,Restaurant,Pizza Place,Other Great Outdoors,Supermarket,Paper / Office Supplies Store,Gym,Big Box Store
8,East York,3,Pizza Place,Pharmacy,Fast Food Restaurant,Gym / Fitness Center,Coffee Shop,Park,Office,Grocery Store,Bakery,Bank
10,North York,3,Coffee Shop,Grocery Store,Bank,Fast Food Restaurant,Caribbean Restaurant,Restaurant,Park,Gas Station,Pizza Place,Sandwich Place
14,East York,3,Pizza Place,Coffee Shop,Park,Pharmacy,Thai Restaurant,Breakfast Spot,Bank,Sandwich Place,Gastropub,Café
17,Etobicoke,3,Baseball Field,Convenience Store,Coffee Shop,Grocery Store,Bank,Pizza Place,Stadium,Restaurant,Café,Discount Store
18,Scarborough,3,Pizza Place,Breakfast Spot,Juice Bar,Fast Food Restaurant,Bank,Coffee Shop,Sandwich Place,Grocery Store,Greek Restaurant,Burger Joint
22,Scarborough,3,Coffee Shop,Pizza Place,Fast Food Restaurant,Bakery,Bank,Indian Restaurant,Sandwich Place,Pharmacy,Beer Store,Discount Store


### Cluster 5

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Etobicoke,4,Pharmacy,Bank,Grocery Store,Supermarket,Park,Café,Liquor Store,Golf Course,Bakery,Bus Line
11,Etobicoke,4,Hotel,Sandwich Place,Park,Convenience Store,Transportation Service,Mexican Restaurant,Farmers Market,Skating Rink,Electronics Store,Bank
12,Scarborough,4,Gym,Italian Restaurant,Burger Joint,Grocery Store,Gym / Fitness Center,Park,Breakfast Spot,Hotel,Dance Studio,Dumpling Restaurant
28,North York,4,Park,Coffee Shop,Bank,Pizza Place,Gas Station,Restaurant,Bus Line,Dog Run,Sandwich Place,Fried Chicken Joint
39,North York,4,Bank,Park,Trail,Gas Station,Intersection,Chinese Restaurant,Café,Restaurant,Skating Rink,Grocery Store
50,North York,4,Park,Bakery,Electronics Store,Asian Restaurant,Vietnamese Restaurant,Italian Restaurant,Sports Bar,Skating Rink,Shopping Mall,Latin American Restaurant
51,Scarborough,4,Park,Harbor / Marina,Sandwich Place,Pizza Place,Pharmacy,Beach,Discount Store,Fast Food Restaurant,Ice Cream Shop,Coffee Shop
58,Scarborough,4,Park,Furniture / Home Store,Restaurant,General Entertainment,Gym,Gym Pool,Café,Diner,Golf Course,Thai Restaurant
87,Downtown Toronto,4,Park,Coffee Shop,Café,Gym,Harbor / Marina,Brewery,Restaurant,Boat or Ferry,Track,Bar
88,Etobicoke,4,Park,Grocery Store,Café,American Restaurant,Breakfast Spot,Bakery,Indian Restaurant,Supermarket,Bar,Thai Restaurant
